In [1]:
library("Repitools")
library(GenomicRanges)
library(gprofiler2)
library(GenomicFeatures)
library(ArchR)
library(SuscrofaTxdb.11.102.fixed)

Le chargement a nécessité le package : BiocGenerics


Attachement du package : ‘BiocGenerics’


Les objets suivants sont masqués depuis ‘package:stats’:

    IQR, mad, sd, var, xtabs


Les objets suivants sont masqués depuis ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Le chargement a nécessité le package : stats4

Le chargement a nécessité le package : S4Vectors


Attachement du package : ‘S4Vectors’


Les objets suivants sont masqués depuis ‘package:base’:

    expand.grid, I, unname


Le chargement a nécessité le package : IRanges

Le chargement a nécessité le package : GenomeInfoDb

Le chargement a nécessité le package : A

In [2]:
txdb_double <- makeTxDbFromGFF("/home/adufour/work/genome_and_annotation/plus4pigs.mapped_to_reference.2023_01_18.Sus_scrofa.Sscrofa11.1.108.sorted.gtf", format="gtf")

Import genomic features from the file as a GRanges object ... 
OK

Prepare the 'metadata' data frame ... 
OK

Make the TxDb object ... 
Warning message in .get_cds_IDX(mcols0$type, mcols0$phase):
“The "phase" metadata column contains non-NA values for features of type
  stop_codon. This information was ignored.”
OK



In [3]:
transcripts_list <- transcripts(txdb_double, columns=c("tx_id", "tx_name","gene_id","tx_type"))

In [4]:
df <- annoGR2DF(transcripts_list)

In [5]:
df$gene_id <- unlist(df$gene_id)

In [6]:
df$strand <- as.character(df$strand)

In [7]:
df["strand"][df["strand"] == "+"] <- 1
df["strand"][df["strand"] == "-"] <- -1

In [8]:
df$tx_type <- "protein_coding"

In [9]:
df <- df[,c("chr", "start", "strand", "gene_id", "tx_type")]

In [10]:
colnames(df) <- c("Chromosome", "Start", "Strand", "Gene", "Transcript_type")

In [11]:
head(df)

,Chromosome,Start,Strand,Gene,Transcript_type
,<fct>,<int>,<chr>,<chr>,<chr>
1,1,889,1,ENSSSCG00000048769,protein_coding
2,1,23676,1,ENSSSCG00000027257,protein_coding
3,1,23784,1,ENSSSCG00000027257,protein_coding
4,1,199358,1,ENSSSCG00000027726,protein_coding
5,1,540929,1,ENSSSCG00000004009,protein_coding
6,1,556911,1,ENSSSCG00000030155,protein_coding


In [12]:
write.csv(df,"/home/adufour/work/scenic_omics/tss.csv", row.names = FALSE)

In [13]:
transcripts_list_2 <- transcripts(txdb_double, columns=c("tx_id", "tx_name","gene_id","tx_type"))

In [14]:
gf_genes <- GenomicFeatures::genes(txdb_double)
gene <- annoGR2DF(gf_genes)

In [15]:
tss_df <- annoGR2DF(transcripts_list_2)

In [16]:
tss_df$tss <- 0
tss_df$tss <- ifelse(tss_df$strand == "+", tss_df$start, tss_df$end)

In [17]:
tss_df$gene_id <- unlist(tss_df$gene_id)

In [18]:
tss_df <- tss_df[,c("chr", "strand", "gene_id", "tx_type", "tss")]

In [19]:
gene_start = function(x){
  A = x[3]
   
  # return product
  return(gene$start[gene$gene_id == A])
}

gene_end = function(x){
  A = x[3]
   
  # return product
  return(gene$end[gene$gene_id == A])
}

In [20]:
tss_df$Start <- apply(tss_df, 1, gene_start)
tss_df$End <- apply(tss_df, 1, gene_end)

In [21]:
tss_df <- tss_df[,c("chr", "Start", "End", "strand", "gene_id", "tss", "tx_type")]

In [22]:
colnames(tss_df) <- c("Chromosome", "Start", "End", "Strand", "Gene", "Transcription_Start_Site", "Transcript_type")

In [23]:
head(tss_df)

,Chromosome,Start,End,Strand,Gene,Transcription_Start_Site,Transcript_type
,<fct>,<int>,<int>,<fct>,<chr>,<int>,<chr>
1,1,889,3866,+,ENSSSCG00000048769,889,transcript
2,1,23676,40032,+,ENSSSCG00000027257,23676,transcript
3,1,23676,40032,+,ENSSSCG00000027257,23784,transcript
4,1,199358,211346,+,ENSSSCG00000027726,199358,transcript
5,1,540929,552882,+,ENSSSCG00000004009,540929,transcript
6,1,556911,576708,+,ENSSSCG00000030155,556911,transcript


In [24]:
write.csv(tss_df,"/home/adufour/work/scenic_omics/tss_scp.csv", row.names = FALSE)

In [25]:
genomeAnnotation <- createGenomeAnnotation(SuscrofaTxdb.11.102.fixed, standard = FALSE)

Getting genome..

Attempting to infer chromSizes..

Attempting to infer blacklist..

Blacklist not downloaded! Continuing without, be careful for downstream biases..



In [26]:
genome_df <- annoGR2DF(genomeAnnotation@listData$chromSizes)

In [27]:
genome_df <- genome_df[,c("chr", "start", "end")]

In [28]:
colnames(genome_df) <- c("Chromosome", "Start", "End")

In [29]:
write.csv(genome_df,"/home/adufour/work/scenic_omics/chromsize.csv", row.names = FALSE)